In [41]:
from analyze_csv import *
import numpy as np
import pandas as pd

In [2]:
results = load_all_files('../results-1pl/')

generalized_dunn_43-generated_2dim_30cl-one_plus_lambda_all_moves.txt
generalized_dunn_13-liver_disorders-one_plus_lambda_all_moves.txt
calinski_harabaz-generated_10dim_30cl-one_plus_lambda_all_moves.txt
generalized_dunn_53-generated_2dim_10cl-one_plus_lambda_all_moves.txt
generalized_dunn_41-glass-one_plus_lambda_all_moves.txt
generalized_dunn_13-generated_2dim_30cl-one_plus_lambda_all_moves.txt
dunn-oil_spill-one_plus_lambda_all_moves.txt
generalized_dunn_51-vehicle-one_plus_lambda_all_moves.txt
generalized_dunn_13-generated_10dim_10cl-one_plus_lambda_all_moves.txt
generalized_dunn_41-generated_2dim_10cl-one_plus_lambda_all_moves.txt
generalized_dunn_41-user_knowledge-one_plus_lambda_all_moves.txt
generalized_dunn_13-oil_spill-one_plus_lambda_all_moves.txt
dunn-heart_statlog-one_plus_lambda_all_moves.txt
davies_bouldin-oil_spill-one_plus_lambda_all_moves.txt
generalized_dunn_51-glass-one_plus_lambda_all_moves.txt
dunn-generated_10dim_30cl-one_plus_lambda_all_moves.txt
generalized_dun

generalized_dunn_51-generated_2dim_30cl-one_plus_lambda_all_moves.txt
generalized_dunn_13-generated_2dim_10cl-one_plus_lambda_all_moves.txt


In [12]:
for i, t in enumerate(results):
    if t[3].iloc[-1]['n_clusters'] > 35:
        print((i,) + t[:3])

(0, 'generalized_dunn_43', 'generated_2dim_30cl', 'one_plus_lambda_all_moves')


In [11]:
results = [r for r in results if r[3].iloc[-1]['n_clusters'] < 50]
len(results)

133

In [24]:
for r in results:
    r[3]['mutation_name'] = [s[:s.find(':')] if s is not None else s for s in r[3]['detail']]
    print(np.unique(np.bincount(r[3].index[1:])))

[ 0 24]
[ 0 24]
[ 0 15 16 24]
[ 0 14 15 16 20 22 24]
[ 0 14 15 16 24]
[ 0 15 16 20]
[ 0 24]
[ 0 14 15 16 24]
[ 0 14 15 16 19 21 22 24]
[ 0 14 15 16 20 23]
[ 0 16 21 23 24]
[ 0 14 15 16 24]
[ 0 14 15 16 19 24]
[ 0 24]
[ 0 14 15 16 19 20]
[ 0 24]
[ 0 24]
[ 0 14 15 16 24]
[ 0 24]
[ 0 15 16 21]
[ 0 14 15 16]
[ 0 12 13 14 15 16 24]
[ 0 14 15 16 17 18 19 20 21 22 23 24]
[ 0 14 15 16 20 22 24]
[ 0 20 22 23 24]
[ 0 24]
[ 0 14 15 16 21 22 23 24]
[ 0 11 12 13 14 15 16 19 20 24]
[ 0 19 20 21 22 23 24]
[ 0 15 16 24]
[ 0 14 15 16 21 24]
[ 0 14 15 16 21]
[ 0 15 19 20 21 22 23 24]
[ 0 15 16 23]
[ 0 16 20 21 22 23 24]
[ 0 14 15 16 24]
[ 0 24]
[ 0 11 12 13 14 15 16 24]
[ 0 14 15 16 20 24]
[ 0 15 16 21]
[ 0 12 14 15 16]
[ 0 15 16 24]
[ 0 24]
[ 0 24]
[ 0 14 15 16 19 24]
[ 0 14 15 16 24]
[ 0 10 11 12 13 14 15 16 19 20 21 22 23]
[ 0 24]
[ 0 12 13 14 15 16]
[ 0 24]
[ 0 24]
[ 0 13 14 15 16 23]
[ 0 13 14 15 16 22 24]
[ 0 14 15 16 21 22 23 24]
[ 0 13 14 15 16 19]
[ 0 15 16 19 21 23 24]
[ 0 13 14 15 16 24]
[ 0 

In [27]:
mutation_names = np.unique(results[0][3]['mutation_name'])[1:]
mutation_names

array(['centroid_hill_climbing_move', 'expand_cluster_move',
       'guided_eliminate_move(density_based_sparseness_separation)',
       'guided_eliminate_move(density_based_validity_separation)',
       'guided_eliminate_move(diameter_separation)',
       'guided_eliminate_move(mean_centroid_distance_separation)',
       'guided_merge_gene_move(centroid_distance_cohesion)',
       'guided_merge_gene_move(density_based_separation_cohesion)',
       'guided_remove_and_reclassify_move(density_based_sparseness_separation)',
       'guided_remove_and_reclassify_move(density_based_validity_separation)',
       'guided_remove_and_reclassify_move(diameter_separation)',
       'guided_remove_and_reclassify_move(mean_centroid_distance_separation)',
       'guided_split_gene_move(density_based_sparseness_separation)',
       'guided_split_gene_move(density_based_validity_separation)',
       'guided_split_gene_move(diameter_separation)',
       'guided_split_gene_move(mean_centroid_distance_sepa

In [30]:
for t in results:
    mut_indices = np.argwhere(t[3]['mutation_name'][:, None] == mutation_names[None, :])[:, 1]
    assert len(mut_indices) == len(t[3]) - 1
    t[3]['mutation_index'] = np.hstack([[-1], mut_indices])

In [54]:
tables = []
for r in results:
    index_series = [t['index'].rename(mutation_names[i] + '_index') for i, t in r[3][['mutation_index', 'index']].groupby('mutation_index')]
    tables.append(pd.concat(index_series, axis=1).values)

In [69]:
mut_indices = []
for r in tables:
    max_indices = np.nanargmax(r, axis=1)
    max_values = np.nanmax(r, axis=1)
    change_steps = np.argwhere(max_values[1:] > max_values[:-1]).flatten() + 1
    mut_indices.append(max_indices[change_steps] - 1)

In [76]:
mut_flags = []
for r in mut_indices:
    flags = np.zeros(len(mutation_names), dtype='bool')
    flags[r] = True
    mut_flags.append(flags)
mut_flags = np.array(mut_flags)

In [85]:
mut_flags.sum(axis=0)

array([71, 37, 45, 43, 36, 29, 23, 21, 29, 32, 37, 26, 20, 25, 28, 30, 19,
       26, 55, 32, 24, 18, 30, 12])